In [1]:
# Install torch geometric
import os
if 'IS_GRADESCOPE_ENV' not in os.environ:
  !pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-2.0.1%2Bcu118.html
  !pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-2.0.1%2Bcu118.html
  !pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-2.0.1%2Bcu118.html
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.1%2Bcu118.html


In [2]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch_geometric
torch_geometric.__version__

'2.3.1'

In [4]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import torch
from collections import defaultdict
from typing import Any, Dict, Iterable, List, Optional, Tuple, Union

import scipy.sparse
from torch import Tensor
from torch.utils.dlpack import from_dlpack, to_dlpack

from torch_geometric.utils.num_nodes import maybe_num_nodes

In [5]:
train=pd.read_csv('/Users/akshaj.g/Desktop/ml/social network /FacebookRecruiting/train.csv')

In [6]:
edge_index = torch.tensor(train.values).T
num_nodes = edge_index.max().item()
print(num_nodes)

1862220


In [7]:
edge_index.shape

torch.Size([2, 9437519])

In [8]:
def to_scipy_sparse_matrix(
    edge_index: Tensor,
    edge_attr: Optional[Tensor] = None,
    num_nodes: Optional[int] = None,
) -> scipy.sparse.coo_matrix:
    row, col = edge_index.cpu()

    if edge_attr is None:
        edge_attr = torch.ones(row.size(0))
    else:
        edge_attr = edge_attr.view(-1).cpu()
        assert edge_attr.size(0) == row.size(0)

    N = maybe_num_nodes(edge_index, num_nodes)
    out = scipy.sparse.coo_matrix(
        (edge_attr.numpy(), (row.numpy(), col.numpy())), (N, N))
    return out


In [9]:
adj_mat=to_scipy_sparse_matrix(edge_index)

In [10]:
!pip3 install tensordict
import torch.nn as nn
from tensordict import TensorDict
from tensordict.nn import TensorDictModule, TensorDictSequential

In [11]:
node_features=defaultdict(torch.Tensor)

In [12]:
def node_degree(edge_list, adj_mat, num_nodes):
  degree=torch.tensor(adj_mat.sum(axis=1).A1, dtype=torch.long)
  degree = torch.where(degree > 0, degree, torch.zeros_like(degree))
  if degree.shape[0] < num_nodes:
    degree = torch.cat((degree, torch.zeros(num_nodes - degree.shape[0], dtype=torch.long)))

  return degree


In [13]:
degree_t=node_degree(edge_index,adj_mat,num_nodes)

In [14]:
import torch
import torch_scatter
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

from torch import Tensor
from typing import Union, Tuple, Optional
from torch_geometric.typing import (OptPairTensor, Adj, Size, NoneType,
                                    OptTensor)

from torch.nn import Parameter, Linear
from torch_sparse import SparseTensor, set_diag
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax

class GNNStack(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, args, emb=False):
        super(GNNStack, self).__init__()
        conv_model = self.build_conv_model(args.model_type)
        self.convs = nn.ModuleList()
        self.convs.append(conv_model(input_dim, hidden_dim))
        assert (args.num_layers >= 1), 'Number of layers is not >=1'
        for l in range(args.num_layers-1):
            self.convs.append(conv_model(args.heads * hidden_dim, hidden_dim))


        self.post_mp = nn.Sequential(
            nn.Linear(args.heads * hidden_dim, hidden_dim), nn.Dropout(args.dropout),
            nn.Linear(hidden_dim, output_dim))

        self.dropout = args.dropout
        self.num_layers = args.num_layers

        self.emb = emb

    def build_conv_model(self, model_type):
        if model_type == 'GraphSage':
            return GraphSage
        elif model_type == 'GAT':
            return GAT
        
    def forward(self, x, edge_index):
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.post_mp(x)

        if self.emb:
            return x

        return F.log_softmax(x, dim=1)
    
    def loss(self, pred, label):
        return F.nll_loss(pred, label)

In [15]:
class GraphSage(MessagePassing):
    
    def __init__(self, in_channels, out_channels, normalize = True,
                 bias = False, **kwargs):  
        super(GraphSage, self).__init__(**kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.normalize = normalize

        self.lin_l = None
        self.lin_r = None
        self.lin_l = nn.Linear(self.in_channels, self.out_channels)
        self.lin_r = nn.Linear(self.in_channels, self.out_channels)
        self.reset_parameters()

    def reset_parameters(self):
        self.lin_l.reset_parameters()
        self.lin_r.reset_parameters()

    def forward(self, x, edge_index, size = None):
        """"""

        out = None
        prop=self.propagate(edge_index, x=(x, x), size=size)
        out=self.lin_l(x) + self.lin_r(prop)
        if self.normalize:
            out = F.normalize(out, p=2)
        return out

    def message(self, x_j):

        out = None
        out = x_j
        return out

    def aggregate(self, inputs, index, dim_size=None):
        out = None
        node_dim = self.node_dim
        out = torch_scatter.scatter(inputs, index, dim=node_dim, reduce='mean')
        return out


In [16]:
class GAT(MessagePassing):

    def __init__(self, in_channels, out_channels, heads = 2,
                 negative_slope = 0.2, dropout = 0., **kwargs):
        super(GAT, self).__init__(node_dim=0, **kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.heads = heads
        self.negative_slope = negative_slope
        self.dropout = dropout

        self.lin_l = None
        self.lin_r = None
        self.att_l = None
        self.att_r = None

        self.lin_l = nn.Linear(self.in_channels, self.heads*self.out_channels)
        self.lin_r = self.lin_l
        self.att_l = nn.Parameter(torch.zeros(self.heads, self.out_channels))
        self.att_r=self.att_l
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.lin_l.weight)
        nn.init.xavier_uniform_(self.lin_r.weight)
        nn.init.xavier_uniform_(self.att_l)
        nn.init.xavier_uniform_(self.att_r)

    def forward(self, x, edge_index, size = None):

        H, C = self.heads, self.out_channels
        x_l=self.lin_l(x).view(-1, H, C)
        x_r=self.lin_r(x).view(-1, H, C)
        alpha_l=self.att_l*x_l
        alpha_r=self.att_r*x_r
        out=self.propagate(edge_index, x=(x_l,x_r), alpha=(alpha_l, alpha_r), size=size).view(-1,H*C)
        return out


    def message(self, x_j, alpha_j, alpha_i, index, ptr, size_i):
        alpha=(alpha_j + alpha_i).squeeze(-1)
        alpha=F.leaky_relu(alpha, negative_slope=0.2)
        if ptr:
            alpha = F.softmax(alpha,ptr)
        else:
            alpha=torch_geometric.utils.softmax(alpha, index)
        alpha=F.dropout(alpha, p=self.dropout, training=self.training)
        out=x_j * alpha
        return out


    def aggregate(self, inputs, index, dim_size = None):
        node_dim = self.node_dim
        out = torch_scatter.scatter(inputs, index, node_dim, dim_size=dim_size, reduce='sum')
        return out

In [17]:
positive_edges=edge_index

In [18]:
test=pd.read_csv('/Users/akshaj.g/Desktop/ml/social network /FacebookRecruiting/test.csv')

In [19]:
test_index = torch.tensor(test.values).T
test_index.shape
edge_index.shape

torch.Size([2, 9437519])

In [20]:
from torch_geometric.utils import negative_sampling

In [21]:
neg_edges=negative_sampling(edge_index)

In [22]:
neg_edges.shape

torch.Size([2, 9437519])

In [23]:
edge_label = torch.tensor(([1.] * 100 + [0.] * 100),requires_grad=True)
edge_label.shape

torch.Size([200])

In [24]:
max_node_index = max(positive_edges.max(), neg_edges.max())
num_nodes = max_node_index + 1
x = torch.rand(num_nodes, 16)

In [25]:
loss_fn = nn.BCELoss()
sigmoid = nn.Sigmoid()

def accuracy(pred, label):
  accu = 0.0
  y_hat = (pred>0.5).type(torch.LongTensor)
  accu = torch.mean((label==y_hat).type(torch.FloatTensor))
  return accu

In [26]:
import torch.optim as optim

def build_optimizer(args, params):
    weight_decay = args.weight_decay
    filter_fn = filter(lambda p : p.requires_grad, params)
    if args.opt == 'adam':
        optimizer = optim.Adam(filter_fn, lr=args.lr, weight_decay=weight_decay)
    elif args.opt == 'sgd':
        optimizer = optim.SGD(filter_fn, lr=args.lr, momentum=0.95, weight_decay=weight_decay)
    elif args.opt == 'rmsprop':
        optimizer = optim.RMSprop(filter_fn, lr=args.lr, weight_decay=weight_decay)
    elif args.opt == 'adagrad':
        optimizer = optim.Adagrad(filter_fn, lr=args.lr, weight_decay=weight_decay)
    if args.opt_scheduler == 'none':
        return None, optimizer
    elif args.opt_scheduler == 'step':
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=args.opt_decay_step, gamma=args.opt_decay_rate)
    elif args.opt_scheduler == 'cos':
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=args.opt_restart)
    return scheduler, optimizer

In [27]:
import time
import networkx as nx
import numpy as np
import torch
import torch.optim as optim
from tqdm import trange
import pandas as pd
import copy

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

import torch_geometric.nn as pyg_nn

import matplotlib.pyplot as plt


def train(positive_edges,neg_edges,args,batch_size):
#building the model 

    model = GNNStack(16, args.hidden_dim, 8,args)
    
    scheduler, opt = build_optimizer(args, model.parameters())

#starting the training 

    for epoch in trange(args.epochs, desc="Training", unit="Epochs"):
        total_loss = 0
        model.train()
        
#shuffle the edges
        p_idx = torch.randperm(positive_edges.size(1))
        p=positive_edges[:,p_idx]
        p=torch.tensor(p)
        n_idx = torch.randperm(neg_edges.size(1))
        n=neg_edges[:,n_idx]
        n=torch.tensor(n)
#batch the edges
        p_b = p[:,:batch_size]
        n_b = n[:,:batch_size]
        train_edge = torch.cat([p_b,n_b], dim=0).T
        print(train_edge.shape)
        opt.zero_grad()
#logic:if they are connected by a edge the dot product=1 else 0 
        emb = model.forward(x,edge_index)
        positive = sigmoid(torch.bmm(emb[train_edge[:, 0]].unsqueeze(1), emb[train_edge[:, 1]].unsqueeze(2)).squeeze())
        negative = sigmoid(torch.bmm(emb[train_edge[:, 2]].unsqueeze(1), emb[train_edge[:, 3]].unsqueeze(2)).squeeze())
        edge_feature = torch.cat([positive , negative])
        print(edge_feature.shape)
        edge_label = torch.cat([torch.ones(batch_size), torch.zeros(batch_size)], dim=0)
        print(edge_label.shape)
#loss is binary cross loss 
            
        loss = loss_fn(edge_feature,edge_label)
        loss.backward()
        opt.step()

        acc = accuracy(edge_feature , edge_label)
        print(emb)
        print(f"Epoch: {epoch+1}, Loss: {loss.item():.4f}, Accuracy: {acc}")
    node_embeddings = emb.detach()
    return node_embeddings

In [28]:
class objectview:
    def __init__(self, d):
        self.__dict__ = d

In [29]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    for args in [
        {'model_type': 'GraphSage', 'num_layers': 2, 'heads': 1, 'batch_size': 32, 'hidden_dim': 32, 'dropout': 0.5, 'epochs': 500, 'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0, 'weight_decay': 5e-3, 'lr': 0.1},
    ]:
        args = objectview(args)
        for model in ['GraphSage']:
            args.model_type = model

            # Match the dimension.
            if model == 'GAT':
                args.heads = 2
            else:
                args.heads = 1
                
trained_node_embeddings = train(positive_edges, neg_edges, args,10000)


Training:   0%|          | 0/500 [00:00<?, ?Epochs/s]/var/folders/_2/2n3n9_3x6ml82z7y8mx8sgbm0000gn/T/ipykernel_50982/1056206024.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  p=torch.tensor(p)
/var/folders/_2/2n3n9_3x6ml82z7y8mx8sgbm0000gn/T/ipykernel_50982/1056206024.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  n=torch.tensor(n)


torch.Size([10000, 4])
torch.Size([20000])
torch.Size([20000])


Training:   0%|          | 1/500 [00:06<56:14,  6.76s/Epochs]

tensor([[-1.9799, -2.0754, -2.3426,  ..., -1.9023, -2.2094, -1.8673],
        [-2.2212, -2.1008, -2.1321,  ..., -2.0227, -2.2648, -1.8507],
        [-1.9951, -2.1122, -2.1703,  ..., -1.9658, -2.2631, -2.0127],
        ...,
        [-2.0607, -1.9907, -2.1835,  ..., -2.0536, -2.3820, -1.9321],
        [-1.9092, -2.2172, -2.3584,  ..., -1.8885, -2.2034, -1.8373],
        [-1.9944, -2.1621, -2.2311,  ..., -2.0330, -2.2518, -1.9404]],
       grad_fn=<LogSoftmaxBackward0>)
Epoch: 1, Loss: 50.0000, Accuracy: 0.5
torch.Size([10000, 4])
torch.Size([20000])
torch.Size([20000])


Training:   0%|          | 2/500 [00:12<50:35,  6.09s/Epochs]

tensor([[-2.0068, -1.9740, -2.0351,  ..., -2.1459, -2.1291, -2.1017],
        [-2.0338, -1.9735, -2.0647,  ..., -2.1223, -2.1161, -2.1251],
        [-2.0506, -1.9997, -2.0748,  ..., -2.1300, -2.1046, -2.1412],
        ...,
        [-2.0619, -2.0056, -2.0686,  ..., -2.1198, -2.1213, -2.1162],
        [-2.0650, -1.9791, -2.0438,  ..., -2.1196, -2.1376, -2.1418],
        [-2.0724, -1.9900, -2.0584,  ..., -2.1117, -2.1244, -2.0870]],
       grad_fn=<LogSoftmaxBackward0>)
Epoch: 2, Loss: 50.0000, Accuracy: 0.5
torch.Size([10000, 4])
torch.Size([20000])
torch.Size([20000])


Training:   1%|          | 3/500 [00:17<47:43,  5.76s/Epochs]

tensor([[-2.1803, -2.0448, -1.9853,  ..., -2.1439, -2.0567, -2.1076],
        [-2.1319, -2.0756, -2.0105,  ..., -2.1221, -2.0512, -2.1071],
        [-2.1591, -2.0810, -2.0054,  ..., -2.1365, -2.0087, -2.1035],
        ...,
        [-2.1704, -2.0770, -2.0171,  ..., -2.1130, -2.0531, -2.0953],
        [-2.1680, -2.0597, -2.0160,  ..., -2.1461, -2.0302, -2.1133],
        [-2.1795, -2.0524, -1.9847,  ..., -2.1484, -2.0412, -2.1505]],
       grad_fn=<LogSoftmaxBackward0>)
Epoch: 3, Loss: 50.0000, Accuracy: 0.5
torch.Size([10000, 4])


Training:   1%|          | 3/500 [00:20<57:02,  6.89s/Epochs]


KeyboardInterrupt: 